Importacion de librerias

pip install aspose-pdf
pip install nltk

In [2]:
#from ast import pattern
from posixpath import split
from PyPDF2 import PdfReader
from pdfreader import SimplePDFViewer
from pandas import ExcelWriter
from pandas import ExcelFile
import unicodedata
import aspose.pdf as rt
import re


Funcion para eliminar simbolos especiales

In [3]:
def limpiar_texto(texto):
    nuevo_texto = texto.lower()
    nuevo_texto = re.sub('http\S+', ' ', nuevo_texto)
    regex = '[\\!\\"\\#\\$\\%\\&\\\'\\(\\)\\*\\+\\,\\-\\.\\/\\:\\;\\<\\=\\>\\?\\@\\[\\\\\\]\\^_\\`\\{\\|\\}\\~]'
    nuevo_texto = re.sub(regex , ' ', nuevo_texto)
    nuevo_texto = re.sub("\d+", ' ', nuevo_texto)
    nuevo_texto = re.sub("\\s+", ' ', nuevo_texto)
    nuevo_texto = nuevo_texto.split(sep = ' ')
    nuevo_texto = [token for token in nuevo_texto if len(token) > 2]
    nuevo_texto=(re.sub(r"([' '][A-Zx][' '])|([A-Z]['//','--'][A-Z])|(([A-Z]{2,10})|([0-9]*['..',',,','--']+[0-9]*)|([0-9]*))","",str((str(nuevo_texto).replace('\n','')).replace('  ',''))))
    return(nuevo_texto)

Cnexion a la base de datos mysql para uardar la informacion leida de las tesis 
installamos prevamente las siguientes liibrerias
pip install mysql-conector-python
pip install python-dotenv

In [4]:
import mysql
import mysql.connector as mysql

print(dir(mysql))
conexion = mysql.connect(
    host = "localhost",
    user = "root",
    password = "Soysuficiente2043@",
    database="tesis"
)

['BINARY', 'Binary', 'CMySQLConnection', 'CharacterSet', 'ClientFlag', 'Connect', 'DATETIME', 'DataError', 'DatabaseError', 'Date', 'DateFromTicks', 'Error', 'FieldFlag', 'FieldType', 'HAVE_CEXT', 'IntegrityError', 'InterfaceError', 'InternalError', 'MySQLConnection', 'NUMBER', 'NotSupportedError', 'OperationalError', 'PoolError', 'ProgrammingError', 'ROWID', 'RefreshOption', 'STRING', 'Time', 'TimeFromTicks', 'Timestamp', 'TimestampFromTicks', 'Warning', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '__version__', '__version_info__', 'abstracts', 'apilevel', 'authentication', 'charsets', 'connect', 'connection', 'connection_cext', 'constants', 'conversion', 'cursor', 'cursor_cext', 'custom_error_exception', 'custom_types', 'dbapi', 'errorcode', 'errors', 'locales', 'logger', 'network', 'opentelemetry', 'optionfiles', 'paramstyle', 'plugins', 'pooling', 'protocol', 'threadsafety', 'tls_ciphers', 'types',

funcion para quitar las stopword 

In [5]:
def eliminar_stopwords(texto):
    stopwords =['agradecimiento','que','contenido','resumen','presentacion','capitulo','generalidades','investigacion','tabla','introduccion','planteamiento','problema',' del ',' los ',' las ',' tabla ',' sin ',' con ',' asi ','declaracion','certificacion','agradecimientos','indice','tablas','dedicatoria','figuras','abreviaturas','viii',' i ',' ii ','iii','xii',' vii ',' xxi ','abstract',' s ',' x ',' xx ',' xix ',' xvi ',' este ',' esta ',' estos ',' estas ','capitulo']
    for i in stopwords:
        texto=texto.replace(i,' ')
    return texto


Codigo generado para
Leer los archivos PDF desde la ruta
Se lee cada hoja del archivo para extraer el director y el capitulo 1 y captulo 2 
Se crea el archivo chi para guardar los resultados de los campos requeridos en este caso
director, key words, abstract
se usa la libreria unicocodedata para remlazar las vocales con tildes en vicales simples

In [ ]:
import re
import os
import pandas as pd
import mysql.connector
ruta='C:/Users/USUARIO/Desktop/IRMA/MAESTRIA-EPN/TESIS/desarrollo/prueba2/'
contenido=os.listdir(ruta)
naux=0;
flg_cap='n'
intDir=''
intC='' 
intCap1=''   
intC2=''
intCap2=''
intR=''
intRes=''
mycursor = conexion.cursor()
sql = "INSERT INTO detalle (director,capitulo_uno,capitulo_dos,resumen,nombre_arch) VALUES (%s,%s,%s,%s,%s)"
listCap=[]
tot=len(contenido)
j=0
for i in range(tot):
    pdfFile=ruta+contenido[i]
    reader=PdfReader(pdfFile)
    numPages=len(reader.pages)
    print(contenido[i])
    for n in range(numPages):
     if n==0:
            page1=reader.pages[n] 
            page2_source=page1.extract_text()
            page1_split=re.split(r"\n",page2_source)
            for i in range(len(page1_split)):
                aux=page1_split[i].rsplit(":")
                aux1=aux[0].lower()
                valor=str(aux1).find('direct')
                if(len(aux)==2 and valor == 0):
                    aux_final=aux;
                    s3 = unicodedata.normalize("NFKD", aux_final[1]).encode("ascii","ignore").decode("ascii")
                    #f.write(s3)
                    intDir=s3 
                    #f.write(";") 
     else:  
            print(n)
            mio=reader.pages[n]
            mio2=mio.extract_text()
            if "RESUMEN" in mio2 and "DEDICATORIA" not in mio2 and "ABSTRACT" not in mio2:
                    s4=unicodedata.normalize("NFKD", mio2).encode("ascii","ignore").decode("ascii")
                    aux=limpiar_texto(s4)
                    aux2=(re.sub(r"([' '][A-Zx][' '])|([A-Z]['//','--'][A-Z])|([A-Z]{2,10})|([0-9]*['..',',,','--']+[0-9]*)|([0-9]*)|('//')|(['\\'][a-z])|['[',']']","",aux))
                    intR=intR+((str(str(aux2).replace('\n',''))).replace('[','').replace(']',''))
                    intRes=eliminar_stopwords(intR)
                    naux=n;
                    print(naux)
            
            else :
                rev=mio2.lower()
                if "capítulo 1" in rev and "índice de contenido" not in rev  :
                    flg_cap ="y"
                    s4=unicodedata.normalize("NFKD", mio2).encode("ascii","ignore").decode("ascii")
                    aux=limpiar_texto(s4)
                    aux2=(re.sub(r"([' '][A-Zx][' '])|([A-Z]['//','--'][A-Z])|([A-Z]{2,10})|([0-9]*['..',',,','--']+[0-9]*)|([0-9]*)|('//')|(['\\'][a-z])|['[',']']","",aux))
                    intC=intC+((str(str(aux2).replace('\n',''))).replace('[','').replace(']',''))
                    #print('antes'+intC)
                    intCap1=eliminar_stopwords(intC)
                else:
                    if "capítulo 3" in rev: 
                        flg_cap="n"
                        n=numPages
                    else:
                        if flg_cap=="y":
                            s4=unicodedata.normalize("NFKD", mio2).encode("ascii","ignore").decode("ascii")
                            aux=limpiar_texto(s4)
                            aux2=(re.sub(r"([' '][A-Zx][' '])|([A-Z]['//','--'][A-Z])|([A-Z]{2,10})|([0-9]*['..',',,','--']+[0-9]*)|([0-9]*)|('//')|(['\\'][a-z])|['[',']']","",aux))
                            intC2=intC2+((str(str(aux2).replace('\n',''))).replace('[','').replace(']',''))
                            intCap2=eliminar_stopwords(intC2)
            if n==naux :
              break            
    if intDir=='':
        intDir='sn'
    if intCap1=='':
        intCap1='sn'
    if intCap2=='':
        intCap2='sn'
    if intRes=='':
        intRes='sn'
        #print(intRes)
    listCap.append(intDir)                    
    listCap.append(intCap1)                    
    listCap.append(intCap2)                    
    listCap.append(intRes)
    arch=contenido[j]
    listCap.append(arch)  
    mycursor.execute(sql, listCap)
    conexion.commit()
    print(mycursor.rowcount, "record inserted.") 
    listCap.clear()
    flg_cap='n'
    intDir=''
    intCap1=''
    intCap2=''
    intRes=''
    intC=''
    intC2=''
    intR=''
    arch=''
    naux=0
    j=j+1
       



Arch_1019_7068.pdf
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
1 record inserted.
Arch_1019_7069.pdf
1
2
3
4
5
6
7
8
9
10
11
12
13
13
1 record inserted.
Arch_1019_7070.pdf
1
2
3
4
5
6
7
8
9
10
11
12
13
13
1 record inserted.


PdfReadError: EOF marker not found